In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import re

url = 'https://store.steampowered.com/search/?sort_by=Released_DESC&tags=-1&category1=998&page=1'

In [2]:
r = requests.get(url)

In [3]:
bs = BeautifulSoup(r.text, 'html.parser')

In [4]:
games = bs.findAll('div', 'responsive_search_name_combined')
print("Found {} games on the web page\n".format(len(games)))

Found 25 games on the web page



In [5]:
for game in games:
    title = game.find('span', 'title').contents[0]
    if not game.find('div', 'col search_released responsive_secondrow').contents:
        release_date = 'Unknown'
    else:
        release_date = game.find('div', 'col search_released responsive_secondrow').contents[0]
    if game.find('div', 'col search_discount responsive_secondrow').find('span') is not None:
        discount = game.find('div', 'col search_discount responsive_secondrow').find('span').contents[0]
        usual_price = game.find('div', 'col search_price discounted responsive_secondrow').find('span').find('strike').contents[0]
        discounted_price = game.find('div', 'col search_price discounted responsive_secondrow').contents[3]
    else:
        discount = None
        discounted_price = None
        usual_price = ' '.join(game.find('div', 'col search_price responsive_secondrow').contents[0].split())
    
    #print ('Title: {}\nRelease date: {}\nDiscount: {}\nUsual price: {}\nDiscounted price: {}\n'.format(title,release_date, discount, usual_price, discounted_price))

In [6]:
def get_game_data(game):
    item = {}
    title = game.find('span', 'title').contents[0]
    if not game.find('div', 'col search_released responsive_secondrow').contents:
        release_date = 'Unknown'
    else:
        release_date = game.find('div', 'col search_released responsive_secondrow').contents[0]
    if game.find('div', 'col search_discount responsive_secondrow').find('span') is not None:
        discount = game.find('div', 'col search_discount responsive_secondrow').find('span').contents[0]
        usual_price = game.find('div', 'col search_price discounted responsive_secondrow').find('span').find('strike').contents[0]
        discounted_price = ' '.join(game.find('div', 'col search_price discounted responsive_secondrow').contents[3].split())
    else:
        discount = None
        discounted_price = None
        usual_price = ' '.join(game.find('div', 'col search_price responsive_secondrow').contents[0].split())
    item = {
        'title': title,
        'release_date': release_date,
        'discount': discount,
        'usual_price': usual_price,
        'discounted_price': discounted_price
    }
    return item

def crawl_page(startswith):
    url = 'https://store.steampowered.com/search/?sort_by=Released_DESC&tags=-1&category1=998&page={}'.format(startswith)
    r = requests.get(url)
    bs = BeautifulSoup(r.text, 'html.parser')
    data, item = [], {}
    games = bs.findAll('div', 'responsive_search_name_combined')
    for game in games:
        item = get_game_data(game)
        data.append(item)
    return data

In [71]:
nb_games_per_page = 25
nb_games = bs.find('div', 'responsive_page_template_content').find('script').contents[0]
nItemCount = nb_games.find('nItemCount')
nb_games = nb_games[nItemCount:]
nb_games = nb_games[:nb_games.index(';')]
nb_games = int(nb_games.split()[-1])
nb_pages = int(np.ceil(nb_games / nb_games_per_page))
print(nb_pages)

na_values = ['Free To Play', '', 'Free']
data = crawl_page(1)
df=pd.DataFrame(data)
for i in range (2,50):
    print(i)
    data=crawl_page(i)
    #data_series = pd.Series(data)
    df = df.append(data, ignore_index=True)
df = df.replace('', np.nan)
df = df.replace('Free', np.nan)
df = df.replace('Free To Play', np.nan)

df_2 = df.dropna(subset=['usual_price'])
print(len(df_2), len(df))
print(len(df_2)/len(df)*100)

1675
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
890 1225
72.6530612244898
